In [2]:
pip install reservoirpy

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir
import numpy as np
from scipy.sparse import random as sparse_random

In [4]:
# hyperparameters
reservoir_units = 500
spectral_radius = 0.9
input_scaling = 0.5
connectivity = 0.1

In [5]:
def preprocess_mnist(batch_size=64):
    transform = transforms.ToTensor()
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [6]:
def initialize_reservoir(input_dim=28, reservoir_size=1000, spectral_radius=0.95, sparsity=0.1, gamma=1.0):
    # Generate a sparse random matrix
    W = sparse_random(reservoir_size, reservoir_size, density=sparsity, format="csr", data_rvs=np.random.randn).A

    # Scale the weights to set the spectral radius
    eigenvalues = np.linalg.eigvals(W)
    W *= spectral_radius / np.max(np.abs(eigenvalues))

    # Initialize the reservoir with the custom weights
    reservoir = Reservoir(
        reservoir_size,
        input_dim=input_dim,
        activation="tanh"
    )
    reservoir.W = W  # Set custom internal weights
    reservoir.gamma = gamma  # Set scaling factor for inputs

    return reservoir


In [7]:
# Compute n-th percentiles from V_tilde
def compute_percentiles(V_tilde_train, n=50):
    return np.percentile(np.abs(V_tilde_train), n, axis=0)

In [8]:
# Apply threshold to reservoir activities V_tilde (to get variable x)
def apply_threshold(V_tilde, percentiles, theta_tilde):
    abs_V_tilde = np.abs(V_tilde)
    thresholds = percentiles + theta_tilde
    x = np.sign(V_tilde) * np.maximum(abs_V_tilde - thresholds, 0)
    return x

In [9]:
# Train output layer - linear
class OutputLayer(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

In [10]:
# Train model SpaRCe ESN
def train_SpaRCe(train_loader, reservoir, ridge_regressor, epochs=10):
    from sklearn.linear_model import Ridge
    import numpy as np
    import torch

    # Initialize data structures for collecting features and labels
    all_features = []
    all_labels = []

    print(f"Starting training for {epochs} epochs...\n")
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        for batch_idx, (images, labels) in enumerate(train_loader):
            # Present images column by column
            batch_features = []
            for image in images:  # Each image
                sequence = image.view(28, 28).T  # Present column by column
                reservoir_output = reservoir.run(sequence.numpy())
                # Apply the SpaRCe transformation here (if applicable)
                tilde_v = reservoir_output.flatten()
                batch_features.append(tilde_v)

            # Stack features and store labels
            all_features.append(np.array(batch_features))
            all_labels.extend(labels.numpy())

            if batch_idx % 10 == 0:  # Print every 10 batches
                print(f"  Processed batch {batch_idx + 1}/{len(train_loader)}")

    # Concatenate all features and labels
    X_train = np.concatenate(all_features, axis=0)
    y_train = np.array(all_labels)

    print("\nTraining Ridge Regressor...")
    ridge_regressor.fit(X_train, y_train)
    print("Training completed!")
    return ridge_regressor


In [11]:
# Evaluate the model
def evaluate(test_loader, reservoir, output_layer, percentiles, theta_tilde):
    correct = 0
    total = 0

    for images, targets in test_loader:
        images = images.squeeze(1)
        batch_states = []

        for t in range(28):
            column = images[:, :, t].numpy()
            state = reservoir.run(column)
            batch_states.append(state)

        reservoir_states = np.hstack(batch_states)
        sparse_states = apply_threshold(reservoir_states, percentiles, theta_tilde)

        sparse_states_tensor = torch.tensor(sparse_states, dtype=torch.float32)
        outputs = output_layer(sparse_states_tensor)
        _, predicted = torch.max(outputs.data, 1)

        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [12]:
# Testing the model
from sklearn.linear_model import Ridge

train_loader, test_loader = preprocess_mnist()
reservoir = initialize_reservoir()
ridge_regressor = Ridge(alpha=1.0)  # Regularization parameter can be tuned
output_layer = train_SpaRCe(train_loader, reservoir, ridge_regressor)
evaluate(test_loader, reservoir, output_layer)


Starting training for 10 epochs...

Epoch 1/10


Running Reservoir-0: 100%|██████████| 28/28 [00:00<00:00, 1743.58it/s]


  Processed batch 1/938


Running Reservoir-0:   0%|          | 0/28 [00:00<?, ?it/s]

KeyboardInterrupt: 